# <b>A. <span style='color:#0B2F9F'><code>SUB-QUERY</code></span></b>
Subquery adalah sebuah query (pernyataan SQL) yang terletak di dalam query lain. Subquery dapat digunakan untuk mengambil data yang kemudian akan digunakan oleh query utama.<br><br>

#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel customer_demography, hitung rata-rata umur customer dan bulatkan ke dua tempat desimal!</b></i>

In [ ]:
SELECT
  ROUND(AVG(age),2) as avg_age
FROM telco_churn.customer_demography

avg_age
46.51


#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel customer_demography, tampilkan customer_id, age dan gendernya dimana umur customer tersebut lebih dari rata-rata umur customer secara keseluruhan!</b></i>

In [ ]:
SELECT
  customer_id,
  age,
  gender
from `telco_churn.customer_demography`
where age > (SELECT ROUND(AVG(age),2) as avg_age FROM telco_churn.customer_demography)

customer_id,gender,age,senior_citizen,married,number_of_dependents
4827-LTQRJ,Female,70,Yes,Yes,2
8809-XKHMD,Male,62,No,Yes,0
3768-NLUBH,Male,73,Yes,Yes,0
8807-ARQET,Female,59,No,No,0
5546-BYZSM,Female,52,No,No,0
...,...,...,...,...,...


#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel customer_demography, ada berapa customer yang lebih dari rata-rata umur customer secara keseluruhan?</b></i>

In [ ]:
SELECT
  COUNT(1) as total_customer
from `telco_churn.customer_demography`
where age > (SELECT ROUND(AVG(age),2) as avg_age FROM telco_churn.customer_demography)

ALIAS

SELECT
  COUNT(age > (SELECT ROUND(AVG(age),2) as avg_age FROM telco_churn.customer_demography)) as total_customer
from `telco_churn.customer_demography`
where age > (SELECT ROUND(AVG(age),2) as avg_age FROM telco_churn.customer_demography)

total_customer
3452


#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel customer_demography, berapa usia termuda dan tertua customer yang lebih dari rata-rata umur customer secara keseluruhan?</b></i>

In [ ]:
SELECT
  MIN(age) as min_age,
  MAX(age) as max_age
from `telco_churn.customer_demography`
where age > (SELECT ROUND(AVG(age),2) as avg_age FROM telco_churn.customer_demography)

min_age,max_age
47,80


#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel customer_status, tampilkan semua data dengan cltv kurang dari atau sama dengan rata-rata cltv customer secara keseluruhan dan statusnya selain Churned?</b></i>

In [ ]:
SELECT
  *
from `telco_churn.customer_status`
where cltv <= (SELECT ROUND(AVG(cltv),2) as avg_age FROM telco_churn.customer_status)

customer_id,satisfaction_score,status,churn_label,churn_score,cltv,churn_category,churn_reason
6867-ACCZI,3,Stayed,No,47,3303,nan,nan
3016-KSVCP,3,Stayed,No,62,3664,nan,nan
3948-FVVRP,4,Stayed,No,73,2848,nan,nan
6047-SUHPR,3,Stayed,No,58,3093,nan,nan
6522-OIQSX,4,Stayed,No,23,4175,nan,nan
...,...,...,...,...,...,...,...


#### <i><b><span style='color:#55679C'>Quest</span> : Pada tabel customer_status, Hitung besarnya presentase setiap status yang ada dan juga jumlah customer tiap statusnya!</b></i>

In [ ]:
SELECT
  status,
  count(customer_id) as total_customer,
  ROUND(count(customer_id)/(SELECT COUNT(customer_id) from `telco_churn.customer_status`)*100,2) as percentage
from `telco_churn.customer_status`
group by status

status,total_customer,percentage
Churned,1869,26.54
Joined,454,6.45
Stayed,4720,67.02


# <b>B. <span style='color:#0B2F9F'><code>CTE</code></span></b>
CTE, atau Common Table Expression, adalah fitur dalam SQL yang memungkinkan Anda untuk mendefinisikan sebuah query sementara yang dapat digunakan dalam query lain. CTE membantu dalam membuat query lebih mudah dibaca dan dipelihara, serta memungkinkan penggunaan rekursi dalam query.

<img src="https://learnsql.com/blog/sql-common-table-expression-guide/cte_syntax_example.webp" width="50%">

#### <i><b><span style='color:#55679C'>Quest</span> : Hitung jumlah customer pada masing - masing offer & jenis contract!</b></i>

In [ ]:
SELECT
  offer,
  contract,
  count(customer_id) as total_customer,
from `telco_churn.telco_services`
group by offer, contract

offer,contract,total_customer
Offer A,Month-to-Month,15
Offer A,One Year,97
Offer A,Two Year,408
Offer B,Month-to-Month,216
Offer B,One Year,317
Offer B,Two Year,291
Offer C,Month-to-Month,203
Offer C,One Year,143
Offer C,Two Year,69
Offer D,Month-to-Month,434


#### <i><b><span style='color:#55679C'>Quest</span> : Tampilkan kolom offer & jenis contract dimana jumlah customernya paling besar (saja)!</b></i>

In [ ]:
WITH count_cust AS(
SELECT
  offer,
  contract,
  count(customer_id) as total_customer,
from `telco_churn.telco_services`
group by offer, contract)

SELECT * from count_cust
WHERE total_customer = (SELECT MAX(total_customer) from count_cust)


offer,contract,total_customer
nan,Month-to-Month,2020


#### <i><b><span style='color:#55679C'>Quest</span> : Tampilkan customer_id, gender, age, tenure_in_month, total_revenue dan status dengan status = 'Churned' dan internet_type-nya = 'Fiber Optic'! (Terapkan CTE untuk mengoptimasi query ini)!</b></i>

In [ ]:
WITH ChurnedCustomerInfo AS (
    SELECT
        cs.customer_id,
        cd.gender,
        cd.age,
        cs.status
    FROM
        telco_churn.customer_status cs
    JOIN
        telco_churn.customer_demography cd ON cs.customer_id = cd.customer_id
    WHERE
        cs.status = 'Churned'
)
SELECT
    cci.customer_id,
    cci.gender,
    cci.age,
    ts.tenure_in_months,
    ts.total_revenue,
    cci.status
FROM
    ChurnedCustomerInfo cci
JOIN
    telco_churn.telco_services ts ON cci.customer_id = ts.customer_id
WHERE
    ts.internet_type = 'Fiber Optic';

customer_id,gender,age,tenure_in_months,total_revenue,status
7495-OOKFY,Female,74,8,1024.1,Churned
1658-BYGOY,Male,71,18,1910.88,Churned
4598-XLKNJ,Female,78,25,2995.07,Churned
4846-WHAFZ,Female,80,37,3102.36,Churned
4412-YLTKF,Female,72,27,2235.41,Churned
...,...,...,...,...,...
